# Solving HNP for attacking ECDSA -- an introduction

In this notebook we present the application of the algorithmic solution of the Hidden Number Problem (HNP) in the context of breaking ECDSA given signatures with partial information on the corresponding nonces.

First, we need some codes to handle Elliptic Curves Cryptographic standards.

### Python codes for ECDSA

For the ECC computation kernel, we use the python codes provided by the [*libecc* project](https://github.com/ANSSI-FR/libecc). So make sure that you downloaded the file *expand_libecc.py* in the same directory of this notebook. You may also need to solve dependencies, some *pip install* should be sufficient (one can obtain *fpylll* by installing  *Sage*).

In [ ]:
import random
from expand_libecc import *
from fpylll import LLL, IntegerMatrix

First, we generate the targeted elliptic curve and the keypair for ECDSA. 

In [ ]:
name = 'secp521r1'
curr_curve_p = 0x01ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff
curr_curve_a = 0x01fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffc
curr_curve_b = 0x0051953eb9618e1c9a1f929a21a0b68540eea2da725b99b315f3b8b489918ef109e156193951ec7e937b1652c0bd3bb1bf073573df883d2c34f1ef451fd46b503f00
curr_curve_n = 0x01fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffa51868783bf2f966b7fcc0148f709a5d03bb5c9b8899c47aebb6fb71e91386409
(curr_curve_gx, curr_curve_gy) = (0x00c6858e06b70404e9cd9e3ecb662395b4429c648139053fb521f828af606b4d3dbaa14b5e77efe75928fe1dc127a2ffa8de3348b3c1856a429bf97e7e31c2e5bd66, 0x011839296a789a3bc0045c8a5fb42c7d1bd998f54449579b446817afbd17273e662c97ee72995ef42640c550b9013fad0761353c7086a272c24088be94769fd16650)

curve_secp521r1 = Curve(curr_curve_a, curr_curve_b, curr_curve_p, curr_curve_n, 1, curr_curve_gx, curr_curve_gy, curr_curve_n, name, None)

keyPair = genKeyPair(curve_secp521r1)

From this keypair one can generate a signature given a message and its corresponding nonce.  

In [ ]:
def sig_ecdsa(hash, keyPair, m, k):
    curve_n = keyPair.pubkey.curve.n
    (sig, kt) = ecdsa_sign(hash, keyPair, m, k)
    assert (kt == k)
    r = stringtoint(sig[0:int(len(sig)/2)])
    s = stringtoint(sig[int(len(sig)/2):])
    h,_,_ = hash(m)
    h = stringtoint(h) % curve_n
    return (s,r,h,k)

Note that signature function is only for demonstration, in particular it outputs the corresponding nonce that should be kept secret. This private value will be used in the following to check the success of our solving process.

### From signatures to linear relations

Any (EC)DSA signature computed from a given ```keyPair``` and a nonce $k$ verifies the following relation
$$s = k^{-1}(h+x\cdot r) \mod n$$

The computation is done modulo $n$ (the order of the curve group), it is a public value that can be obtained from ```keyPair.pubkey.curve.n```. The integer $x$ corresponds to the secret key which is used to generate all the  signatures. One can see that $x$ can be easily obtained form any nonce $k$ and the corresponding signature:
$$ x = r^{-1}(k\cdot s - h) \mod n$$
This is the reason why all the nonces should be kept secret and chosen very carefully. Thus it is clear now that attacking ECDSA is equivalent to find a nonce corresponding to a message since one can deduce $x$. Let see now how one can retrieve such a secret value when some signatures are computed from small nonces. 

From two signatures, one can obtain a system of two modular linear relations (all the relatoin are implicitly considered  modulo $n$). 
$$
\begin{cases} s_0 = k_0^{-1}(h_0+x\cdot r_0)\\ s_1 = k_1^{-1}(h_1+x\cdot r_1) \end{cases}
$$
The main algebraic problem here is the number of unknown values in front of the number of relations: 3 over 2. Thus we can not solve it by regular linear solving (e.g. Gauss reduction). A way to solve this problem is to chase common unknown between the relations and eliminate them. Go to elimininate! 

By respectively multiplying by $k_i$ one obtains the following relations 
$$
\begin{cases} h_0+x\cdot r_0 - s_0\cdot k_0 =0\\ h_1+x\cdot r_1 - s_1\cdot k_1 = 0\end{cases}
$$
Now by cross multiplying the relation of index $1$ by $r_0$ and the one of index $0$ by $r_1$ the same value $x\cdot r_0 \cdot r_1$ appears in the two relations:
$$
\begin{cases} r_1\cdot h_0+x\cdot r_0\cdot r_1 - r_1\cdot s_0\cdot k_0 =0\\ r_0\cdot h_1+x\cdot r_0\cdot r_1 - r_0\cdot s_1\cdot k_1 = 0\end{cases}
$$


One can subtract the two relations to eliminate the common value and finally obtain a relation which gives $k_1$ as a relation of $k_0$:
$$k_1 + t_1\cdot k_0 + u_1 = 0$$
where $t_1 = -r_0^{-1}\cdot s_1^{-1}\cdot r_1\cdot s_0$ and $u_1 = r_0^{-1}\cdot s_1^{-1}\cdot r_1\cdot h_0 - s_1^{-1}\cdot h_1$.

The problem seems to be the same as before since we have one relation for two unknown values $k_1$ and $k_0$. This is true, but in the context where these two unknown values are *small* in comparison to the modulo $n$, one can use some lattice reduction in order to solve the problem. The fact that $x$ is supposed to be of full size (the same as $n$) and the $k_i$'s small is the reason why we chose to elimate $x$.

### Solving the case of two unknowns

We now have to solve a linear relation in the two integer variables $k_0$ and $k_1$:
$$k_1 + t_1\cdot k_0 + u_1 = 0 \mod n \Leftrightarrow k_1 + t_1\cdot k_0 + u_1 + k\cdot n = 0$$
To proceed, let see our solution as a vector $(-k_1, k_0, X)$ in a *lattice* (one can think of it as a discrete vector space generated by an integer matrix):   
$$ (k, k_0, 1) \cdot
\left(\begin{matrix}
n   &  0 & 0 \\
t_1 &  1 & 0 \\
u_1 &  0 & X \\
\end{matrix}\right)
= (-k_1, k_0, X)
$$
The integer $X$ is chosen as an integer of the size of the $k_i$ in order to somehow control the elements in the lattice in a way that they are all at least of this size and make our targeted vector exceptionaly small.  

It is important to note that the matrix generating the lattice can be computed from known values obtained from signatures. For example, one can use the following function:

In [ ]:
def lin_relation(sig1, sig2, curve_n):
# return (t,u) the linear relation of sig1 in function of sig2
# k1 + t*k2 + u = 0 mod curve_n    
    (s1, r1, h1, _) = sig1
    (s2, r2, h2, _) = sig2
    
    Ir2 = pow(r2,-1, curve_n)
    Is1 = pow(s1,-1, curve_n) 

    t = - Ir2*Is1*r1*s2
    u = - Is1*h1 + Ir2*Is1*r1*h2

    return (t,u)

Then, the matrix can be computed from two signatures:

In [ ]:
k0 = 0x4141b7d438041bdd95b4653885691681e69f46a4aa56a22393771fbd7b648ad
m0 = 'SSTIC22'+str(0)
sig0 = sig_ecdsa(sha512, keyPair, m0, k0)

k1 = 0xc1aba7823a44da3cdeda1eeda39d41c2d7e4fc9bae06a372ec0aff52200c44
m1 = 'SSTIC22'+str(1)
sig1 = sig_ecdsa(sha512, keyPair, m1, k1)

(t1,u1) = lin_relation(sig1, sig0, keyPair.pubkey.curve.n)

X = 2**(k0.bit_length())

mat = IntegerMatrix(3, 3)
mat[0,0] = keyPair.pubkey.curve.n
mat[1,0] = t1
mat[2,0] = u1
mat[1,1] = 1
mat[2,2] = X

Now we want to find our solution in the lattice generated by the rows of this matrix. To do that, we use an algorithm that produce a basis of the same lattice but with generating vectors which are almost orthogonal and thus of smaller size. Thus, if our solution is sufficiently small, it should appear as one of the line of this new basis. This reduction algorithm is called [LLL](https://en.wikipedia.org/wiki/Lenstra%E2%80%93Lenstra%E2%80%93Lov%C3%A1sz_lattice_basis_reduction_algorithm). 

In [ ]:
LLL.reduction(mat)
print("The targeted vector is in the reduced basis: ",(mat[0,0] == -k1) and (mat[0,1] == k0) and (mat[0,2] == X))

As one can see, our solution vector is retrieved as the first vector of the new basis computed by LLL. It is important to note that in this example, the $k_i$'s are very small in comparison to $n$.

In [ ]:
print('Size of k0, k1:', k0.bit_length(), k1.bit_length())
print('Size of n:', keyPair.pubkey.curve.n.bit_length())

This is the reason why our solution has been found so easily. In a context where the $k_i$'s are not as small, one needs more relations (and thus signatures) in order to find the secret data. 

### The general case

In this context, we assume that we have $s+1$ signatures corresponding to nonces $k_i$'s of supposed size $\ell$ but with a msb of $t$ null bits. As in the case of two signatures, one can form a system of modular linear relation where the secret key is elimated by writing each $k_i$ for $i>1$ as a function on $k_0$. 

$$
\begin{cases}
k_1 + t_1\cdot k_0 + u_1 = 0\\
k_2 + t_2\cdot k_0 + u_2 = 0\\
\vdots\\
k_s + t_s\cdot k_0 + u_s = 0
\end{cases}
$$

From this system, one can form a matrix generating a lattice where a shortest vector allows to retrieve the secret nonces: 

$$ (k^1, k^2,\ldots,k_0, 1) \cdot
\left(\begin{matrix}
n  &  0 & \cdots &&& 0 \\
0  &  n & \cdots &&& 0 \\
0  &  0 & \ddots &&& 0 \\
\vdots &  &  & &\vdots \\
0 &  0 & \cdots &n& 0 & 0\\
t_1 &  t_2 & \cdots && 1 & 0\\
u_1 &  u_2 & \cdots && 0 & X \\
\end{matrix}\right)
= (-k_1, -k_2,\ldots, k_s,k_0, X)
$$

This matrix $M$ can be computed with the following function 

In [ ]:
def HNP_MSBNULL_Matrix(list_signatures, curve_n, nonces_bound):
    m = len(list_signatures)
    ti = []
    ui = []
    ki = []
    (s0,r0,h0, k0) = list_signatures[0]
    
    for i in range(1,m):
        (si,ri,hi, k ) = list_signatures[i]
        
        (t,u) = lin_relation((si,ri,hi, k), (s0,r0,h0, k0), curve_n)
                
        ti += [t]
        ui += [u]
        ki += [k]
    
    lattice = IntegerMatrix(m + 1, m + 1)
    for i in range(m-1):
        lattice[i,i] = curve_n
        lattice[m-1,i]   = ti[i]
        lattice[m,i] = ui[i]
        
        
    lattice[m-1,m-1] = 1
    lattice[m-1,m] = 0
    lattice[m,m-1] = 0
    lattice[m,m] = nonces_bound
    
    return lattice


In order to obtain an idea about the dimension of the matrix $M$ needed to solve the given problem, one may follow a usual approach based on an assumption. This is based on a theoretical result (proven by Ajtai) which gives a bound on the Euclidean norm of a shortest vector $\nu$ in a random lattice of dimension $d$ generated by a matrix $M$: 
$$
\|\nu\| \leqslant \sqrt{\frac{d}{2\pi e}}\text{Vol}(M)^{1/d}
$$
where $\text{Vol}(M)=\det(M^t M)^{1/2}$ is called the volume of the lattice gnerated by $M$ (note that this quantity is independant of the choice of the lattice's basis).  

This result is called *Gaussiam heuristic* since this assumption is in general true in practice for any lattice of large dimension and thus assumed true also for applications as the one we try to solve. 

In our case, we are focusing on the vector $v$ containing the nonces of the given signatures, we thus build a matrix $M$ such that this inequality becomes true. To do so, one needs to evaluate the volume of $M$ and the norm of the targeted vector $v$. Then, by using an algorithm that reduce the basis $M$ one may hope to retrieve a shortest vector which could reveal the secrets.

In our context, one can easily evaluate the norm of the vector $v$ containing the $s+1$ secrets nonces (recall that in this case the matrix is of dimension $s+2$):
$$
\|v\|^2 \leq (s+1)2^{2(\ell - t)}\cdot X^2 
$$
and it is the same for the matrix $M$ thanks to its triangular shape (recall that X is a bound for the $k_i$'s):
$$
\det(M^t M)^{1/(s+2)} \geq n^{2s/(s+2)}\cdot X^{2/(s+2)} 
$$
From these inequalities one may find the least integer verifying the Gaussian heuristic inequality in the context of our application. 

In [ ]:
l=521
t=9
n = curr_curve_n
s = 2
while True:
    if (s+1) < ((s+2)/18)*(pow(n,2*s/(s+2))*pow(2, 2*(l-t)/(s+2) - 2*(l-t))):
        break
    s = s+1

print("Minimal dimension where Gaussian assumption could be true:",s)



To test this approach, let us now generate some signatures computed from nonces with 9 null msb (this corresponds to our side channel result). To do that, one can use the following function.

In [ ]:
def null_MSb_evil_nonce(size, null_msb):
    evil_nonce = random.getrandbits(size-null_msb)
    return (evil_nonce, null_msb)

def HNP_MSBNULL_List(size, partial_msb, keyPair, mess_base):
    
    curve_n = keyPair.pubkey.curve.n
    part = partial_msb
    m0 = mess_base + str(0)
    k0,_ = null_MSb_evil_nonce(curve_n.bit_length(), part)

    sig0 = sig_ecdsa(sha512,keyPair,m0,k0)
    
    lsig = [sig0]

    for i in range(1,size):
        m = mess_base + str(i)
        k,_ = null_MSb_evil_nonce(curve_n.bit_length(), part)
        sig = sig_ecdsa(sha512,keyPair,m,k)
        (t,u) = lin_relation(sig, sig0, curve_n)

        lsig += [sig]
    return lsig

part = 9 
nbsig = 800
lsig = HNP_MSBNULL_List(nbsig, part, keyPair, 'SSTIC')

Now we can test this process and find the target vector by LLL reduction of the matrix formed by these signatures. 

In [ ]:
curve_n = keyPair.pubkey.curve.n
nbrel = 75
succes = 0
X = 2**(curve_n.bit_length() - part - 1)

for ti in range(10):
    lrel = random.choices(lsig, k=nbrel)
    mat = HNP_MSBNULL_Matrix(lrel, curve_n, X)
    _,_,_,k = lrel[0]

    LLL.reduction(mat)

    test = False
    for i in range(nbrel):
        test = test or ((mat[i,nbrel] == X) and (mat[i,nbrel-1] == k))    
    if test:
        succes += 1

print("Succes proba for {} null MSb solved with {} relations: {}/10".format(part, nbrel, succes))


This success rate is very low but recall that the solving process depends on an assumption and we took the least integer verifying the inequality. More clearly, the theoretical result related to the Gaussian heuristic is proven for random lattices of large dimension. In practice, the success rate of the Gaussian heuistic becomes better when the difference $\sqrt{\frac{d}{2\pi e}}\text{Vol}(M)^{1/d} - \|v\|$ is sufficiently large. In this situation, the vector $v$ would have more chance to become a shortest vector and thus to be catched by a lattice reduction algorithm. To do that, let us consider matrices of larger dimension (almost the double size here). 

In [ ]:
curve_n = keyPair.pubkey.curve.n
nbrel = 110
succes = 0
X = 2**(curve_n.bit_length() - part - 1)

for ti in range(10):
    lrel = random.choices(lsig, k=nbrel)
    mat = HNP_MSBNULL_Matrix(lrel, curve_n, X)
    _,_,_,k = lrel[0]

    LLL.reduction(mat)

    test = False
    for i in range(nbrel):
        test = test or ((mat[i,nbrel] == X) and (mat[i,nbrel-1] == k))    
    if test:
        succes += 1

print("Succes proba for {} null MSb solved with {} relations: {}/10".format(part, nbrel, succes))


Youhou! Thanks to this higher dimensional matrices, we succeded to solve our problem: the target vector containing the secret short nonces has been found in most of the tests we condidered. Then from one of the nonce, one can retrieve the secret key.

The solving process presented here is very introductive and does not consider all the possible tweaks, for example it is possible to change the relying problem by considering the search of a closest vector to a given one. Also, one can obtain a more efficient method by considering another starting matrix, for example by considering *centered remainders*. In this case, one may obtain the same result with a matrix of smaller dimension. One can also use another reduction algorithm as *BKZ* to obtain better results in some cases. But it is important to remenber that all the underlying mathematical problems are known to be hard thus, for all these reduction algortihms, the quality of the solutions tends to dramatically reduce with the increase of the dimension of the input matrix. Said differently, it becomes very hard to solve the same problem when only few bits (e.g. 2 or 3) of the nonces are known.